In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dataclasses
import json
from pathlib import Path
from typing import List 
from urllib.parse import urljoin

import requests

from owmapi import SimpleOWMApi, Station, WFData

In [3]:
API_KEY = Path("data/myfreekey.txt").open().read().strip()

In [4]:
owm = SimpleOWMApi(API_KEY)

In [5]:
stations = Station.from_csv("data/weather_stations.csv")
stations[:3]

[Station(station_id=7005, name='ABBEVILLE', latitude=50.136, longitude=1.834, altitude=69),
 Station(station_id=7015, name='LILLE-LESQUIN', latitude=50.57, longitude=3.0975, altitude=47),
 Station(station_id=7020, name='PTE DE LA HAGUE', latitude=49.725167, longitude=-1.939833, altitude=6)]

In [6]:
class WXplorerSDK:
    base_url = "http://localhost:8000/"

    @classmethod
    def _url(cls, endpoint: str) -> str:
        return urljoin(cls.base_url, endpoint)
    
    @staticmethod
    def _method(meth: str, *args, **kwargs):
        func = getattr(requests, meth)
        resp = func(*args, **kwargs)
        resp.raise_for_status()
        return resp.json()
    
    def post(self, endpoint, data):
        return self._method("post", self._url(endpoint), json=data)
    
    def get(self, endpoint):
        return self._method("get", self._url(endpoint))
    
    def list_stations(self):
        return [Station(**s) for s in self.get("/stations/")]
    
    def get_station_weather(self):
        return self.get("/weather/")
    
    def register_stations(self, stations: List[Station]):
        for s in stations:
            try:
                self.post("/stations/", data=s._asdict())
            except requests.HTTPError as e:
                print(f"Station registering {s.name} failed.\n{e}")
    
    def save_station_forecast(self, s: Station, wdata: List[WFData]):
        print(f"Handling staton {s.name}")
        for forecast in wdata:
            try:
                self.post("/weather/", data={
                    "station": s.station_id,
                    **forecast._asdict()
                })
            except requests.HTTPError as e:
                print(f"Failed to register forcast for station {s.name} (forecast_time: {forecast.forecast_time}). ")
    

In [7]:
def main()
    stations = Station.from_csv("data/weather_stations.csv")
    sdk.register_stations(stations)
    owm = SimpleOWMApi()
    for s in stations:
        station_forecast = owm.forecast(lon=s.longitude, lat=s.latitude)
        sdk.save_station_forecast(s, station_forecast)
main()

In [11]:
station_list = sdk.list_stations()
station_list[:3]

[Station(station_id=7005, name='ABBEVILLE', latitude=50.136, longitude=1.834, altitude=69),
 Station(station_id=7015, name='LILLE-LESQUIN', latitude=50.57, longitude=3.0975, altitude=47),
 Station(station_id=7020, name='PTE DE LA HAGUE', latitude=49.725167, longitude=-1.939833, altitude=6)]